In [1]:
import pandas as pd
import numpy as np
import warnings
import re
warnings.filterwarnings('ignore')

In [ ]:
dta_files = ['MTF_HH_Battery_Data_Final.dta','MTF_HH_Cooking_Data_Final.dta',
             'MTF_HH_Core_Survey_Final_Data_trimmed-2.dta','MTF_HH_Roster.dta',
             'MTF_HH_Sec.M1_Asset_Data_Final.dta','MTF_HH_Sec.M2_Asset_Data_Final.dta',
             'MTF_HH_Sec.M3_Asset_Data_Final.dta','MTF_HH_Solar_Roster.dta','MTF_section_i_to_k.dta']

In [2]:
df = pd.read_stata(r'raw_data/dta/MTF_HH_Core_Survey_Final_Data_trimmed-2.dta', convert_categoricals = False)

# ACCESS TO GRID CONNECTION (SECTION C)

c_c_2

Is the household connected to the national
grid?

Issues:

c_35 - c_37

In [3]:
grid_connection_map = {1: 'Yes', 2: 'No'}
locality_map = {1:'Urban', 2:'Rural'} #Per-Urban = Rural

In [4]:
def get_percentages(target_column, column_map, target_new_name, by_locality = False):
    
    if by_locality:
        
        target_df = df[['locality_ur', target_column]]
        target_df['locality_ur'] = target_df['locality_ur'].replace(locality_map)
        target_df[target_column] = target_df[target_column].replace(column_map)
        
        target = target_df.groupby(['locality_ur', target_column]).size()
        target =  target.groupby(level=0).apply(lambda x: 100 * x / float(x.sum())).reset_index().round(2)
        target.columns = ['Locality', target_new_name, '% Population']
        
        return target

    else:
        df[target_column] = df[target_column].replace(column_map)
        target = df[target_column].value_counts(normalize = True)*100
        target = target.to_frame().reset_index().round(2)
        target.columns  = [target_new_name, '% Population']
        
        return target

## Overall

In [5]:
grid_overall = get_percentages('c_c_2', grid_connection_map, 'Grid Connection')
grid_overall

,Grid Connection,% Population
0,No,55.49
1,Yes,44.51


## By Locality

In [6]:
grid_by_locality = get_percentages('c_c_2', grid_connection_map, 'Grid Connection', True)
grid_by_locality

,Locality,Grid Connection,% Population
0,Rural,No,80.05
1,Rural,Yes,19.95
2,Urban,No,23.09
3,Urban,Yes,76.91


## ISSUES

### In the last 12 months, did any of your appliances get damaged because the voltage was going up and down from the grid?

c_c_35

In [7]:
damaged_appliances_map = {1: 'Yes', 2:'No', 888: 'Dont Know' ,  np.nan : 'No Answer'}

In [8]:
damages_overall = get_percentages('c_c_35', damaged_appliances_map, 'Damaged Appliances')
damages_overall

,Damaged Appliances,% Population
0,No Answer,56.23
1,No,34.99
2,Yes,8.34
3,Dont Know,0.44


In [9]:
damages_by_locality = get_percentages('c_c_35', damaged_appliances_map, 'Damaged Appliances', True)
damages_by_locality

,Locality,Damaged Appliances,% Population
0,Rural,Dont Know,0.31
1,Rural,No,15.82
2,Rural,No Answer,80.20
3,Rural,Yes,3.68
4,Urban,Dont Know,0.61
5,Urban,No,60.28
6,Urban,No Answer,24.61
7,Urban,Yes,14.50


### What are the most serious problems you experience with your grid electricity?
c_c_36

First: 'c_c_36afirst'

Second: 'c_c_36bsecond'

In [10]:
grid_problems_map = {1:"Unpredictable interruptions/outages", 2:"Supply shortages", 3:'Voltage fluctuations',
                     4: "High cost of electricity", 59:"Maintenance/customer care response", 
                     555: "Other, specify", 111 : "No problems", np.nan : 'No Answer'}

In [11]:
problems_first = get_percentages('c_c_36afirst', grid_problems_map, 'First Most Problems')
problems_first

,First Most Problems,% Population
0,No Answer,57.08
1,Unpredictable interruptions/outages,20.81
2,No problems,9.37
3,High cost of electricity,7.67
4,Voltage fluctuations,3.31
5,Supply shortages,1.46
6,"Other, specify",0.31


In [12]:
problems_second = get_percentages('c_c_36bsecond', grid_problems_map, 'Second Most Problems')
problems_second

,Second Most Problems,% Population
0,No Answer,66.47
1,No problems,9.63
2,Unpredictable interruptions/outages,7.97
3,High cost of electricity,7.73
4,Voltage fluctuations,4.12
5,Supply shortages,3.75
6,"Other, specify",0.33


### In the last 12 months, did any household members die or have permanent limb (bodily injury) damage because of the grid electricity?

c_c_37

In [13]:
yes_no_map = {1: 'Yes', 2: 'No'}

In [14]:
death_injury_overall =  get_percentages('c_c_37', yes_no_map, 'Death/Injury')
death_injury_overall

,Death/Injury,% Population
0,No,99.05
1,Yes,0.95


In [15]:
death_injury_locality =  get_percentages('c_c_37', yes_no_map, 'Death/Injury', True)
death_injury_locality

,Locality,Death/Injury,% Population
0,Rural,No,99.03
1,Rural,Yes,0.97
2,Urban,No,99.06
3,Urban,Yes,0.94


# ELECTRICITY

## Of all the sources that you mentioned above, which is the source that you use most of the time in your household? This will be the MAIN electricity source that is referred to later

c_c_159

In [16]:
elec_source_map = {1 : "National Grid Connection", 2 : "Local Mini Grid", 3 : "Generator (Connecting one or more households)",
                   4 : "Solar Home System", 5 : "Solar Lantern/Lighting System", 6 : "Rechargeable Battery",
                   7 : "Pico-Hydro", 8 : "No electricity", 555: "Other, specify"}

In [17]:
electricity_source_overall =  get_percentages('c_c_159', elec_source_map, 'Main Electricity Source')
electricity_source_overall

,Main Electricity Source,% Population
0,National Grid Connection,43.88
1,No electricity,26.67
2,Solar Lantern/Lighting System,15.01
3,Solar Home System,7.52
4,"Other, specify",4.70
5,Rechargeable Battery,1.28
6,Local Mini Grid,0.65
7,Generator (Connecting one or more households),0.19
8,Pico-Hydro,0.10


In [18]:
electricity_source_locality =  get_percentages('c_c_159', elec_source_map, 'Main Electricity Source', True)
electricity_source_locality

,Locality,Main Electricity Source,% Population
0,Rural,Generator (Connecting one or more households),0.35
1,Rural,Local Mini Grid,0.09
2,Rural,National Grid Connection,19.81
3,Rural,No electricity,37.38
4,Rural,"Other, specify",6.86
5,Rural,Pico-Hydro,0.17
6,Rural,Rechargeable Battery,1.90
7,Rural,Solar Home System,11.31
8,Rural,Solar Lantern/Lighting System,22.14
9,Urban,Local Mini Grid,1.36


# COOKING (SECTION I)

## Main Stove

In the last 12 months, which cookstove did your household use for preparing meals? List up to 5

i_3

In [19]:
cooking  = pd.read_stata(r'raw_data/dta/MTF_HH_Cooking_Data_Final.dta', convert_categoricals = False)

In [20]:
cooking.columns[:30]

Index(['cluuq', 'HH_code', 'prov', 'cty', 'dist', 'div', 'loc', 'subloc',
       'class', 'locality_urp', 'locality_ur', 'grid_loc', 'i_position',
       'i_3_stove_type', 'i_3_other', 'i_3_label', 'i_4_obtain', 'i_5_cost',
       'i_6_giver', 'i_7_trng', 'i_8a_brand', 'i_8b_model', 'i_8c_resemble',
       'i_9_metal', 'i_10_no_burners', 'i_11_value', 'i_12_time_used',
       'i_13_mm_used1', 'i_13_mm_used2', 'i_13_mm_used3'],
      dtype='object')

In [21]:
stoves = cooking.groupby(['HH_code', 'locality_ur'], as_index = False).apply(lambda x: pd.Series({'HH_Stoves' : list(x['i_3_label']),
                                                'HH_Main_Stove' : list(x['i_3_label'])[0],   
                                                'HH_Stoves_Count' : len(list(x['i_3_label'])),
                                                'HH_Unique_Stoves' : set(x['i_3_label']),
                                                'HH_Unique_Stoves_Count' : len(set(x['i_3_label']))}))
stoves

,HH_code,locality_ur,HH_Stoves,HH_Main_Stove,HH_Stoves_Count,HH_Unique_Stoves,HH_Unique_Stoves_Count
0,1070101041_01,1.0,[Manufactured stove],Manufactured stove,1,{Manufactured stove},1
1,1070101041_02,1.0,[Manufactured stove],Manufactured stove,1,{Manufactured stove},1
2,1070101041_03,1.0,[Manufactured stove],Manufactured stove,1,{Manufactured stove},1
3,1070101041_04,1.0,[Manufactured stove],Manufactured stove,1,{Manufactured stove},1
4,1070101041_05,1.0,[Manufactured stove],Manufactured stove,1,{Manufactured stove},1
...,...,...,...,...,...,...,...
4056,8010705032_10,1.0,[Manufactured stove],Manufactured stove,1,{Manufactured stove},1
4057,8010705032_11,1.0,"[Manufactured stove, Manufactured stove]",Manufactured stove,2,{Manufactured stove},1
4058,8010801011_01,1.0,[Manufactured stove],Manufactured stove,1,{Manufactured stove},1
4059,8010801011_03,1.0,[Manufactured stove],Manufactured stove,1,{Manufactured stove},1


In [22]:
main_stoves = stoves['HH_Main_Stove'].value_counts(normalize = True)*100
main_stoves = main_stoves.to_frame().reset_index().round(2)
main_stoves.columns = ['Stove Type', '% Population']
main_stoves

,Stove Type,% Population
0,Three stone/Open fire,53.34
1,Manufactured stove,39.15
2,Other Self-Built Stove,7.07
3,,0.27
4,other,0.12
5,Other,0.05


In [23]:
stoves['locality_ur'] = stoves['locality_ur'].replace(locality_map)
main_stove_locality = stoves.groupby(['locality_ur', 'HH_Main_Stove']).size()
main_stove_locality =  main_stove_locality.groupby(level=0).apply(lambda x: 100 * x / float(x.sum())).reset_index().round(2)
main_stove_locality.columns = ['Locality', 'HH_Main_Stove', '% Population']
main_stove_locality

,Locality,HH_Main_Stove,% Population
0,Rural,,0.22
1,Rural,Manufactured stove,16.80
2,Rural,Other Self-Built Stove,6.21
3,Rural,Three stone/Open fire,76.55
4,Rural,other,0.22
5,Urban,,0.34
6,Urban,Manufactured stove,68.43
7,Urban,Other,0.11
8,Urban,Other Self-Built Stove,8.19
9,Urban,Three stone/Open fire,22.92


## Stove Stacking

In [24]:
unique_stoves = stoves['HH_Unique_Stoves_Count'].value_counts(normalize = True)*100
unique_stoves = unique_stoves.to_frame().reset_index().round(2)
unique_stoves.columns = ['HH_Unique_Stoves_Count', '% Population']
unique_stoves

,HH_Unique_Stoves_Count,% Population
0,1,86.83
1,2,12.88
2,3,0.30


In [25]:
unique_stove_locality = stoves.groupby(['locality_ur', 'HH_Unique_Stoves_Count']).size()
unique_stove_locality =  unique_stove_locality.groupby(level=0).apply(lambda x: 100 * x / float(x.sum())).reset_index().round(2)
unique_stove_locality.columns = ['Locality', 'HH_Unique_Stoves_Count', '% Population']
unique_stove_locality

,Locality,HH_Unique_Stoves_Count,% Population
0,Rural,1,83.59
1,Rural,2,15.98
2,Rural,3,0.43
3,Urban,1,91.07
4,Urban,2,8.82
5,Urban,3,0.11


## Main Cooking Fuel

i_18 In the last 12 months, what are the fuels you used on this cookstove?


In [26]:
cooking_fuel_map = {1 : "Kerosene", 2 : "Coal/lignite", 3 : "Charcoal", 4 : "Wood", 5 : "Solar", 
                    6 : "Animal Waste/Dung", 7 : "Crop Residue/Plant Biomass", 8 : "Saw Dust",
                    9 : "Coal Briquette", 10 : "Biomass Briquette", 11 : "Processed biomass (pellets)/woodchips",
                    12 : "Ethanol", 13 : "Biogas", 14 : "LPG/cooking gas", 15 : "Piped Natural Gas",
                    16 : "Electricity", 17 : "Garbage/plastic", 18 : "Not applicable", 555 : "Other, specify"}
types_mapping = {'Electric':'Electric/Solar', 'Solar':'Electric/Solar', 'Animal Waste/Dung':'Dung',
                'Crop Residue/Plant Biomass':'Crop residue', 'Pellets/Processes biomass/wood chips':'Processed biomas'}

In [27]:
fuels = cooking.groupby(['HH_code', 'locality_ur'], as_index = False).apply(lambda x: pd.Series({'HH_Fuels' : list(x['i_18_a_1st_fuel']),
                                                'HH_Main_Fuel' : list(x['i_18_a_1st_fuel'])[0],   
                                                'HH_Fuels_Count' : len(list(x['i_18_a_1st_fuel'])),
                                                'HH_Unique_Fuels' : set(x['i_18_a_1st_fuel']),
                                                'HH_Unique_Fuels_Count' : len(set(x['i_18_a_1st_fuel']))}))
fuels

,HH_code,locality_ur,HH_Fuels,HH_Main_Fuel,HH_Fuels_Count,HH_Unique_Fuels,HH_Unique_Fuels_Count
0,1070101041_01,1.0,[1.0],1.0,1,{1.0},1
1,1070101041_02,1.0,[14.0],14.0,1,{14.0},1
2,1070101041_03,1.0,[3.0],3.0,1,{3.0},1
3,1070101041_04,1.0,[3.0],3.0,1,{3.0},1
4,1070101041_05,1.0,[14.0],14.0,1,{14.0},1
...,...,...,...,...,...,...,...
4056,8010705032_10,1.0,[14.0],14.0,1,{14.0},1
4057,8010705032_11,1.0,"[14.0, 3.0]",14.0,2,"{3.0, 14.0}",2
4058,8010801011_01,1.0,[1.0],1.0,1,{1.0},1
4059,8010801011_03,1.0,[1.0],1.0,1,{1.0},1


In [28]:
fuels['HH_Main_Fuel'] = fuels["HH_Main_Fuel"].replace(cooking_fuel_map)

In [29]:
fuels

,HH_code,locality_ur,HH_Fuels,HH_Main_Fuel,HH_Fuels_Count,HH_Unique_Fuels,HH_Unique_Fuels_Count
0,1070101041_01,1.0,[1.0],Kerosene,1,{1.0},1
1,1070101041_02,1.0,[14.0],LPG/cooking gas,1,{14.0},1
2,1070101041_03,1.0,[3.0],Charcoal,1,{3.0},1
3,1070101041_04,1.0,[3.0],Charcoal,1,{3.0},1
4,1070101041_05,1.0,[14.0],LPG/cooking gas,1,{14.0},1
...,...,...,...,...,...,...,...
4056,8010705032_10,1.0,[14.0],LPG/cooking gas,1,{14.0},1
4057,8010705032_11,1.0,"[14.0, 3.0]",LPG/cooking gas,2,"{3.0, 14.0}",2
4058,8010801011_01,1.0,[1.0],Kerosene,1,{1.0},1
4059,8010801011_03,1.0,[1.0],Kerosene,1,{1.0},1


In [30]:
main_fuels = fuels['HH_Main_Fuel'].value_counts(normalize = True)*100
main_fuels = main_fuels.to_frame().reset_index().round(2)
main_fuels.columns = ['Fuel Type', '% Population']
main_fuels

,Fuel Type,% Population
0,Wood,55.99
1,Charcoal,17.78
2,LPG/cooking gas,16.05
3,Kerosene,7.26
4,Not applicable,2.15
5,Biomass Briquette,0.22
6,Crop Residue/Plant Biomass,0.17
7,Electricity,0.17
8,Biogas,0.15
9,Coal/lignite,0.02


In [31]:
fuels['locality_ur'] = fuels['locality_ur'].replace(locality_map)
main_fuel_locality = fuels.groupby(['locality_ur', 'HH_Main_Fuel']).size()
main_fuel_locality =  main_fuel_locality.groupby(level=0).apply(lambda x: 100 * x / float(x.sum())).reset_index().round(2)
main_fuel_locality.columns = ['Locality', 'HH_Main_Fuel', '% Population']
main_fuel_locality

,Locality,HH_Main_Fuel,% Population
0,Rural,Biogas,0.09
1,Rural,Biomass Briquette,0.39
2,Rural,Charcoal,10.76
3,Rural,Crop Residue/Plant Biomass,0.17
4,Rural,Electricity,0.09
5,Rural,Kerosene,2.09
6,Rural,LPG/cooking gas,4.01
7,Rural,Not applicable,1.61
8,Rural,Processed biomass (pellets)/woodchips,0.04
9,Rural,Wood,80.75


## Issues


In the last 12 months, what type of harm/injury did your household face from this cookstove? Multiple responses possible.

I_30

In [32]:
stoves_harm_map = {1 : "Death or permanent damage", 2 : "Burns/fire/poisoning", 
                   3 : "Severe cough/respiratory problem", 4 : "Other major injury", 
                   5 : "Minor injury", 6 : "Fire with no injury", 
                   7 : "Itchy/watery eyes", 8 : "Light cough", 9 : "None/NO"}

In [33]:
def cooking_summary(target_column, column_map, target_new_name, by_locality = False):
    
    target_df = cooking[['HH_code', 'locality_ur', target_column]]
    target_col_map = target_column + '_map'
    target_df[target_col_map] = target_df[target_column].replace(column_map)
    target_df['locality_ur'] = target_df['locality_ur'].replace(locality_map)
    target = target_df.groupby(['HH_code', 'locality_ur'], as_index = False).apply(lambda x: pd.Series({'HH_Col_Values' : list(x[target_col_map]),
                                                'HH_Main_Value' : list(x[target_col_map])[0],   
                                                'HH_Values_Count' : len(list(x[target_col_map])),
                                                'HH_Unique_Values' : set(x[target_col_map]),
                                                'HH_Unique_Values_Count' : len(set(x[target_col_map]))}))
    if by_locality:
        
        main_locality = target.groupby(['locality_ur', 'HH_Main_Value']).size()
        main_locality =  main_locality.groupby(level=0).apply(lambda x: 100 * x / float(x.sum())).reset_index().round(2)
        main_locality.columns = ['Locality', target_new_name, '% Population']
        
        return main_locality
    
    else:
        
        main = target['HH_Main_Value'].value_counts(normalize = True)*100
        main = main.to_frame().reset_index().round(2)
        main.columns = [target_new_name, '% Population']
        
        return main     

In [34]:
main_stoves_issues_overall = cooking_summary('i_30_harm', stoves_harm_map, 'Main_Stove_Issues')
main_stoves_issues_overall

,Main_Stove_Issues,% Population
0,None/NO,81.71
1,0.0,7.44
2,Itchy/watery eyes,3.90
3,Minor injury,2.78
4,Light cough,1.59
5,Burns/fire/poisoning,1.39
6,Severe cough/respiratory problem,0.60
7,Fire with no injury,0.47
8,Other major injury,0.10
9,Death or permanent damage,0.02


In [35]:
main_stoves_issues_locality = cooking_summary('i_30_harm', stoves_harm_map, 'Main_Stove_Issues', True)
main_stoves_issues_locality

,Locality,Main_Stove_Issues,% Population
0,Rural,0.0,8.46
1,Rural,Burns/fire/poisoning,1.62
2,Rural,Fire with no injury,0.61
3,Rural,Itchy/watery eyes,5.26
4,Rural,Light cough,2.02
5,Rural,Minor injury,3.51
6,Rural,None/NO,78.18
7,Rural,Other major injury,0.09
8,Rural,Severe cough/respiratory problem,0.26
9,Urban,0.0,6.12


# FUEL CONSUMPTION (SECTION H)

In the last 12 months, did your household use this energy?

In [36]:
fuels_map = {"a" : "LPG", "b" : "Wood purchased", "c" : "Wood collected", "d" : "Charcoal",
             "e" : "Kerosene", "f" : "Piped Natural Gas", "g" :  "Coal/Lignite",
             "h" : "Animal waste/ Dung", "i" : "Crop Residue/Plant Biomass/Sawdust",
             "j" : "Briquette/Pellet", "k" : "Biogas", "l" : "Ethanol",
             "m" : "Electricity", "n" : "Other, specify"}

In [37]:
fuels_map

{'a': 'LPG',
 'b': 'Wood purchased',
 'c': 'Wood collected',
 'd': 'Charcoal',
 'e': 'Kerosene',
 'f': 'Piped Natural Gas',
 'g': 'Coal/Lignite',
 'h': 'Animal waste/ Dung',
 'i': 'Crop Residue/Plant Biomass/Sawdust',
 'j': 'Briquette/Pellet',
 'k': 'Biogas',
 'l': 'Ethanol',
 'm': 'Electricity',
 'n': 'Other, specify'}

In [38]:
target_h3 = [col for col in df.columns if re.match(r'\bh_h_3\w\b', col)]
target_h3

['h_h_3a',
 'h_h_3b',
 'h_h_3c',
 'h_h_3d',
 'h_h_3e',
 'h_h_3h',
 'h_h_3i',
 'h_h_3k',
 'h_h_3m',
 'h_h_3n']

In [39]:
fuels_columns = ['locality_ur'] + target_h3
fuels_new_columns = ['locality_ur'] +  [fuels_map[col[-1]] for col in target_h3]

In [40]:
fuels_df = df[fuels_columns]
fuels_df.columns = fuels_new_columns

In [41]:
a = fuels_df.fillna('No Response').replace({' ':'No Response',1:'Yes',2:'No', '1':'Yes',"2":'No'})

In [42]:
b = a.melt(id_vars = 'locality_ur', value_vars = fuels_df.columns[1:])
b['locality_ur'] = b['locality_ur'].replace({'Yes':'Urban', 'No': 'Rural'})

In [43]:
b

,locality_ur,variable,value
0,Urban,LPG,Yes
1,Urban,LPG,Yes
2,Urban,LPG,Yes
3,Urban,LPG,Yes
4,Urban,LPG,Yes
...,...,...,...
45895,Urban,"Other, specify",Yes
45896,Urban,"Other, specify",Yes
45897,Urban,"Other, specify",No Response
45898,Urban,"Other, specify",No Response


## Overall

In [44]:
fuel_overall = b.groupby(['variable', 'value']).size()
fuel_overall =  fuel_overall.groupby(level=0).apply(lambda x: 100 * x / float(x.sum())).reset_index().round(2)
fuel_overall.columns = ['Fuel Type', 'Response', '% Population']
fuel_overall

,Fuel Type,Response,% Population
0,Animal waste/ Dung,No Response,99.96
1,Animal waste/ Dung,Yes,0.04
2,Biogas,No Response,99.63
3,Biogas,Yes,0.37
4,Charcoal,No,0.83
5,Charcoal,No Response,70.20
6,Charcoal,Yes,28.98
7,Crop Residue/Plant Biomass/Sawdust,No,0.02
8,Crop Residue/Plant Biomass/Sawdust,No Response,99.13
9,Crop Residue/Plant Biomass/Sawdust,Yes,0.85


## By Locality

In [45]:
fuel_by_locality = b.groupby(['locality_ur', 'variable', 'value']).size()
fuel_by_locality =  fuel_by_locality.groupby(level=[1,0]).apply(lambda x: 100 * x / float(x.sum())).reset_index().round(2)
fuel_by_locality.columns = ['Locality', 'Fuel Type', 'Response', '% Population']
fuel_by_locality

,Locality,Fuel Type,Response,% Population
0,Rural,Animal waste/ Dung,No Response,99.92
1,Rural,Animal waste/ Dung,Yes,0.08
2,Rural,Biogas,No Response,99.69
3,Rural,Biogas,Yes,0.31
4,Rural,Charcoal,No,0.65
5,Rural,Charcoal,No Response,77.90
6,Rural,Charcoal,Yes,21.45
7,Rural,Crop Residue/Plant Biomass/Sawdust,No,0.04
8,Rural,Crop Residue/Plant Biomass/Sawdust,No Response,98.85
9,Rural,Crop Residue/Plant Biomass/Sawdust,Yes,1.11
